# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from scipy.ndimage import affine_transform
from skimage import measure
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

def load_ants_affine_mat(mat_file_path):
    # Load .mat file

    rotation = np.array([
    [0.941498875617981, 0.19202075898647308, -0.08786671608686447], 
    [-0.198630228638649, 0.781607985496521, 0.015325229614973068],
    [0.06243732571601868, 0.0008880351670086384, 0.8622015118598938]
    ]
    )
    translation = np.array([266.45843505859375, 150.64747619628906, -138.71780395507812])
    center = np.array([730.082275390625, 373.0629577636719, 569.5])
    # Apply rotation around center
    affine = np.eye(4)
    affine[:3, :3] = rotation
    affine[:3, 3] = translation + center - rotation @ center

    return affine

def convert_lps_to_sar(affine_lps):
    # Flip L (x) and P (y) axes: LPS to RAS => flip x and y
    flip = np.diag([-1, -1, 1, 1])
    affine_sar = flip @ affine_lps @ flip
    return affine_sar

In [18]:
moving_name = 'MD594'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
#bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

37


In [ ]:
from scipy.io import loadmat
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
transform_file = 'MD594_Allen_10.0x10.0x10.0um_sagittal_to_Allen.mat'
transform_path = os.path.join(reg_path, moving_name, transform_file)
transform = sitk.ReadTransform(transform_path)
transform = sitk.AffineTransform(transform)

In [19]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 8.98433334e-01 -2.65091877e-01  5.95423255e-02 -2.13381393e+01]
 [ 3.65894141e-01  1.13223847e+00  4.76580505e-02 -6.40295926e+02]
 [-5.71300691e-02 -1.34311696e-02  1.17741204e+00  2.81177915e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [20]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS 238.5831606646421
# MD585 to MD589 RMS 18.2658167690059

RMS 25.07982021040849


In [21]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    x,y,z = moving_all[structure]
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)

    #z,y,x = transform.TransformPoint((z,y,x))
    #print(structure, x,y,z)
    #transformed = (x,y,z)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))


RMS 245.55296057283311


In [22]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = affine_transform_point(com, transformation_matrix)
x,y,z = com

print(f'{moving_name} {structure} apply ants {x=}, {y=} {z=}')

print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')


MD594 SC non trans [1119.  385.  517.]
MD594 SC apply ants x=1119.232944161371, y=384.50369985453443 z=516.5074759237448
MD594 SC apply trans [913. 229. 567.]
Allen SC [914. 239. 569.]
MD594->Allen SC [ -1. -10.  -2.]


In [17]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,MD594,Allen,transformed,difference,sumsquares
Index,,,,,,
36,RtTg,"[493.0, 266.0, 210.0]","[380.0, 213.0, 228.0]","[377.0, 235.0, 227.0]","[-3.0, 22.0, -1.0]",22.4673
9,6N_L,"[493.0, 266.0, 210.0]","[431.0, 209.0, 212.0]","[377.0, 235.0, 227.0]","[-54.0, 27.0, 15.0]",62.0021
35,6N_R,"[493.0, 266.0, 210.0]","[431.0, 209.0, 244.0]","[377.0, 235.0, 227.0]","[-54.0, 27.0, -17.0]",62.3822
7,7n_L,"[493.0, 266.0, 210.0]","[427.0, 219.0, 188.0]","[377.0, 235.0, 227.0]","[-51.0, 16.0, 39.0]",65.6043
26,7n_R,"[493.0, 266.0, 210.0]","[428.0, 219.0, 267.0]","[377.0, 235.0, 227.0]","[-51.0, 16.0, -40.0]",66.4696
29,5N_L,"[493.0, 266.0, 210.0]","[408.0, 211.0, 164.0]","[377.0, 235.0, 227.0]","[-31.0, 24.0, 63.0]",74.2642
8,5N_R,"[493.0, 266.0, 210.0]","[408.0, 211.0, 292.0]","[377.0, 235.0, 227.0]","[-31.0, 24.0, -65.0]",75.4905
17,SNC_L,"[493.0, 266.0, 210.0]","[334.0, 205.0, 172.0]","[377.0, 235.0, 227.0]","[43.0, 30.0, 55.0]",76.0790
28,VLL_L,"[493.0, 266.0, 210.0]","[379.0, 207.0, 156.0]","[377.0, 235.0, 227.0]","[-2.0, 28.0, 71.0]",76.7251


In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
def ants_3d_to_scipy_2d(affine_3d, plane='axial', slice_index=0):
    """
    Convert a 3D ANTs affine transformation matrix to a 2D affine transform
    suitable for scipy.ndimage.affine_transform.

    Parameters:
        affine_3d (np.ndarray): A 4x4 affine matrix from ANTs.
        plane (str): Plane to slice through ('axial', 'coronal', 'sagittal').
        slice_index (int): Index of the slice in the chosen plane.

    Returns:
        matrix_2d (np.ndarray): 2x2 affine transformation matrix.
        offset_2d (np.ndarray): Length-2 offset vector.
    """
    if affine_3d.shape != (4, 4):
        raise ValueError("Expected a 4x4 affine transformation matrix.")

    # Extract rotation+scaling and translation components
    rotation_scaling = affine_3d[:3, :3]
    translation = affine_3d[:3, 3]

    if plane == 'axial':
        matrix_2d = rotation_scaling[:2, :2]
        offset_2d = translation[:2] + rotation_scaling[:2, 2] * slice_index
    elif plane == 'coronal':
        matrix_2d = rotation_scaling[[0,2], :][:, [0,2]]
        offset_2d = translation[[0,2]] + rotation_scaling[[0,2], 1] * slice_index
    elif plane == 'sagittal':
        matrix_2d = rotation_scaling[1:3, 1:3]
        offset_2d = translation[1:3] + rotation_scaling[1:3, 0] * slice_index
    else:
        raise ValueError("Plane must be 'axial', 'coronal', or 'sagittal'.")

    return matrix_2d, offset_2d

In [ ]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
from cloudvolume import CloudVolume
tq = LocalTaskQueue(parallel=2)